In [56]:
import json
import pandas
import time
import os
import recordlinkage
import jellyfish as jf
import csv

In [57]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('sourceFiles.json', "r").read())

## Load the file schema
sourceSchema = json.loads(open('sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('idConfig.json', "r").read())

In [58]:
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [59]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)
    

sports247
rivals
cfbRef
allConf
allAmerican
nflData


In [60]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [61]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [62]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)

In [63]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)

#df_sports247.dtypes
#df_rivals.dtypes


In [70]:
# This creates the root of each recruit in the JSON.  
#Anything that can be pulled out from the various sources is here
masterData = []
for recruit in sports247:
    print(recruit)
    masterRecruit = {}
    masterRecruit['ID'] = recruit['ID']
    masterRecruit['year'] = recruit['year']
    masterRecruit['school'] = recruit['school']
    if ('compRating' in recruit.keys()):
        masterRecruit['compositeScore'] = recruit['compRating']
        masterRecruit['compositeStars'] = recruit['compStars']
    masterData.append(masterRecruit)  

{'school': 'nebraska', 'year': '2012', 'playerName': 'paulthurston', 'highSchool': 'Arvada West', 'city': 'Arvada', 'state': 'CO', 'position': 'OT', 'height': 77.0, 'weight': '275', 'compRating': ' 0.9407  ', 'compStars': 4, 'nationalRank': '151', 'positionRank': '15', 'stateRank': '2', '247Rating': ' 92  ', '247Stars': 4, '247positionRank': '26', '247stateRank': '2', 'ID': 'paulthurston_nebraska'}
{'school': 'nebraska', 'year': '2012', 'playerName': 'gregmcmullen', 'highSchool': 'Archbishop Hoban', 'city': 'Akron', 'state': 'OH', 'position': 'SDE', 'height': 77.0, 'weight': '255', 'compRating': ' 0.9099  ', 'compStars': 4, 'nationalRank': '259', 'positionRank': '14', 'stateRank': '18', '247Rating': ' 95  ', '247Stars': 4, '247nationalRank': '139', '247positionRank': '6', '247stateRank': '8', 'ID': 'gregmcmullen_nebraska'}
{'school': 'nebraska', 'year': '2012', 'playerName': 'jordanwesterkamp', 'highSchool': 'Montini Catholic', 'city': 'Lombard', 'state': 'IL', 'position': 'WR', 'heigh

{'school': 'maryland', 'year': '2019', 'playerName': 'dinotomlin', 'highSchool': 'Shady Side Academy', 'city': 'Pittsburgh', 'state': 'PA', 'position': 'WR', 'height': 72.0, 'weight': '165', 'compRating': ' 0.8478  ', 'compStars': 3, 'nationalRank': '1172', 'positionRank': '131', 'stateRank': '23', '247Rating': ' 87  ', '247Stars': 3, '247positionRank': '83', '247stateRank': '10', 'ID': 'dinotomlin_maryland'}
{'school': 'maryland', 'year': '2019', 'playerName': 'deshawnholt', 'highSchool': 'Westlake', 'city': 'Waldorf', 'state': 'MD', 'position': 'WDE', 'height': 77.0, 'weight': '220', 'compRating': ' 0.8474  ', 'compStars': 3, 'nationalRank': '1189', 'positionRank': '73', 'stateRank': '21', '247Rating': ' 85  ', '247Stars': 3, '247positionRank': '76', '247stateRank': '17', 'ID': 'deshawnholt_maryland'}
{'school': 'maryland', 'year': '2019', 'playerName': 'samokuayinonu', 'highSchool': 'Mesabi Range C.C.', 'city': 'Virginia', 'state': 'MN', 'position': 'SDE', 'height': 75.0, 'weight': 

{'school': 'nebraska', 'year': '2003', 'playerName': 'walimuhammad', 'highSchool': 'Coffeyville C.C.', 'city': 'Coffeyville', 'state': 'KS', 'position': 'WDE', 'height': 73.0, 'weight': '235', 'compRating': ' 0.8333  ', 'compStars': 3, 'nationalRank': '94', 'positionRank': '4', 'stateRank': '22', '247Rating': '  N/A  ', '247Stars': 0, '247positionRank': 'N/A', '247stateRank': 'N/A', 'ID': 'walimuhammad_nebraska'}
{'school': 'nebraska', 'year': '2003', 'playerName': 'joedailey', 'highSchool': 'St. Peters Prep', 'city': 'Jersey City', 'state': 'NJ', 'position': 'DUAL', 'height': 72.0, 'weight': '202', 'compRating': ' 0.8333  ', 'compStars': 3, 'nationalRank': '640', 'positionRank': '19', 'stateRank': '26', '247Rating': '  N/A  ', '247Stars': 0, '247positionRank': 'N/A', '247stateRank': 'N/A', 'ID': 'joedailey_nebraska'}
{'school': 'nebraska', 'year': '2003', 'playerName': 'darrendelone', 'highSchool': 'Mt. San Antonio College', 'city': 'Walnut', 'state': 'CA', 'position': 'OT', 'height':

{'school': 'iowa', 'year': '2017', 'playerName': 'tristanwirfs', 'highSchool': 'Mount Vernon', 'city': 'Mount Vernon', 'state': 'IA', 'position': 'OT', 'height': 76.5, 'weight': '290', 'compRating': ' 0.8911  ', 'compStars': 4, 'nationalRank': '331', 'positionRank': '33', 'stateRank': '3', '247Rating': ' 94  ', '247Stars': 4, '247nationalRank': '126', '247positionRank': '15', '247stateRank': '2', 'ID': 'tristanwirfs_iowa'}
{'school': 'iowa', 'year': '2017', 'playerName': 'djimoncolbert', 'highSchool': 'Bishop Miege', 'city': 'Mission', 'state': 'KS', 'position': 'CB', 'height': 72.0, 'weight': '210', 'compRating': ' 0.8590  ', 'compStars': 3, 'nationalRank': '643', 'positionRank': '62', 'stateRank': '3', '247Rating': ' 90  ', '247Stars': 4, '247positionRank': '32', '247stateRank': '2', 'ID': 'djimoncolbert_iowa'}
{'school': 'iowa', 'year': '2017', 'playerName': 'peytonmansell', 'highSchool': 'Belton', 'city': 'Belton', 'state': 'TX', 'position': 'DUAL', 'height': 74.0, 'weight': '189',

{'school': 'purdue', 'year': '2019', 'playerName': 'marvingrant', 'highSchool': 'Martin Luther King', 'city': 'Detroit', 'state': 'MI', 'position': 'S', 'height': 74.0, 'weight': '200', 'compRating': ' 0.9041  ', 'compStars': 4, 'nationalRank': '276', 'positionRank': '23', 'stateRank': '8', '247Rating': ' 88  ', '247Stars': 3, '247positionRank': '45', '247stateRank': '13', 'ID': 'marvingrant_purdue'}
{'school': 'purdue', 'year': '2019', 'playerName': 'stevenfaucheux', 'highSchool': 'Lakota West', 'city': 'West Chester', 'state': 'OH', 'position': 'DT', 'height': 76.5, 'weight': '272', 'compRating': ' 0.8891  ', 'compStars': 3, 'nationalRank': '395', 'positionRank': '28', 'stateRank': '14', '247Rating': ' 88  ', '247Stars': 3, '247positionRank': '43', '247stateRank': '19', 'ID': 'stevenfaucheux_purdue'}
{'school': 'purdue', 'year': '2019', 'playerName': 'mershawnrice', 'highSchool': 'Reynoldsburg', 'city': 'Reynoldsburg', 'state': 'OH', 'position': 'ATH', 'height': 74.0, 'weight': '198'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [77]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [78]:
print(len(candidate_links))

22388448


In [79]:
c = recordlinkage.Compare()


c.exact('ID', 'ID', label='ID')

#c.string(str('playerName'), str('playerName'), method='damerau_levenshtein', threshold=5, label='playerName')
c.string('city', 'city', method='levenshtein', threshold=10, label='city')
#c.string('state', 'state', method='levenshtein', threshold=3, label='state')
#c.string('highSchool', 'highSchool', method='levenshtein', threshold=5, label='highSchool')

# The comparison vectors
features = c.compute(candidate_links, df_sports247, df_rivals)

features

ID  city
0     19      0   0.0
      34      0   0.0
      43      0   0.0
      51      0   0.0
      55      0   0.0
...          ..   ...
17063 16853   0   0.0
      16855   0   0.0
      16856   0   0.0
      16861   0   0.0
      16877   0   0.0

[22388448 rows x 2 columns]

In [80]:
features.describe()

,ID,city
count,2.238845e+07,22388448.0
mean,7.146989e-04,0.0
std,2.672430e-02,0.0
min,0.000000e+00,0.0
25%,0.000000e+00,0.0
50%,0.000000e+00,0.0
75%,0.000000e+00,0.0
max,1.000000e+00,0.0


In [81]:
df_sports247['city']

0               Arvada
1                Akron
2              Lombard
3          Gainesville
4               Cibolo
             ...      
17059          Decatur
17060             Yuma
17061            Miami
17062    Bowling Green
17063        Pikeville
Name: city, Length: 17064, dtype: object

In [82]:
df_rivals['city']

0          Albuquerque
1           Scottsdale
2            Charlotte
3          Hyattsville
4        Medford Lakes
             ...      
16888          Madison
16889     Jacksonville
16890        Nashville
16891            Tampa
16892           Summit
Name: city, Length: 16893, dtype: object

In [86]:
count = 0
for record in sports247:
    for rival in rivals:
        if (record['city'] == rival['city']):
            count = count + 1
print(count)

613238


In [94]:
features.sum(axis=1).value_counts().sort_index(ascending=False)
matches = features[features.sum(axis=1) == 1]
type(matches)

pandas.core.frame.DataFrame

In [93]:
type(matches)
matchesDict = matches.to_dict()
type(matchesDict)

In [90]:
print(matchesDict)

{'ID': {(0, 3813): 1, (1, 5477): 1, (2, 4699): 1, (3, 3374): 1, (4, 515): 1, (6, 2851): 1, (7, 2908): 1, (8, 603): 1, (9, 5579): 1, (10, 3890): 1, (11, 5318): 1, (12, 5011): 1, (13, 3690): 1, (14, 3174): 1, (15, 4073): 1, (16, 2729): 1, (17, 748): 1, (18, 5345): 1, (19, 5828): 1, (20, 3520): 1, (21, 4723): 1, (22, 4010): 1, (23, 2707): 1, (24, 2735): 1, (25, 61): 1, (26, 1144): 1, (27, 5574): 1, (28, 4826): 1, (29, 4091): 1, (30, 5463): 1, (31, 2503): 1, (32, 3078): 1, (33, 3215): 1, (34, 3024): 1, (35, 5437): 1, (36, 5218): 1, (37, 1039): 1, (38, 5550): 1, (39, 1368): 1, (40, 5034): 1, (41, 890): 1, (42, 2138): 1, (43, 5151): 1, (44, 422): 1, (45, 1328): 1, (46, 4725): 1, (47, 70): 1, (48, 2128): 1, (51, 1388): 1, (52, 19): 1, (53, 2958): 1, (53, 4213): 1, (54, 25): 1, (55, 5696): 1, (56, 3761): 1, (58, 4844): 1, (59, 3806): 1, (61, 4337): 1, (62, 4003): 1, (64, 3394): 1, (65, 3654): 1, (67, 5015): 1, (68, 4936): 1, (69, 4845): 1, (70, 3568): 1, (71, 4001): 1, (72, 432): 1, (72, 5338)

In [ ]:
for i in matchesDict:
    for j in in:
        

In [126]:
print(ecm)

In [127]:
ecm[0]

TypeError: 'ECMClassifier' object is not subscriptable